# Generate results table

## Super class split, gao model

In [ ]:
# super class c-c
       epoch/time_per_step 0.02734
wandb:                 epoch/training_throughput 18728.15601
wandb:                            epoch/val_loss 12.88653
wandb:                        epoch/val_mol1_acc 0.33696
wandb:                       epoch/val_mol1_loss 2.95378
wandb:                       epoch/val_mol1_top3 0.59264
wandb:                       epoch/val_mol1_top5 0.7178
wandb:                        epoch/val_mol2_acc 0.47445
wandb:                       epoch/val_mol2_loss 2.16868
wandb:                       epoch/val_mol2_top3 0.70842
wandb:                       epoch/val_mol2_top5 0.81954
wandb:                        epoch/val_mol3_acc 0.33771
wandb:                       epoch/val_mol3_loss 3.42702
wandb:                       epoch/val_mol3_top3 0.59837
wandb:                       epoch/val_mol3_top5 0.70646
wandb:                        epoch/val_mol4_acc 0.38303
wandb:                       epoch/val_mol4_loss 2.93002
wandb:                       epoch/val_mol4_top3 0.65406
wandb:                       epoch/val_mol4_top5 0.76928
wandb:                        epoch/val_mol5_acc 0.75338
wandb:                       epoch/val_mol5_loss 1.40703
wandb:                       epoch/val_mol5_top3 0.85342
wandb:                       epoch/val_mol5_top5 0.9122
wandb:   frequency_informed_agent_accuracy_top_1 0.08994
wandb:   frequency_informed_agent_accuracy_top_3 0.20701
wandb: frequency_informed_overall_accuracy_top_1 0.01809
wandb: frequency_informed_overall_accuracy_top_3 0.04406
wandb: frequency_informed_solvent_accuracy_top_1 0.08589
wandb: frequency_informed_solvent_accuracy_top_3 0.22972
wandb:                                      loss 9.44101
wandb:                                  mol1_acc 0.35059
wandb:                                 mol1_loss 2.24691
wandb:                                 mol1_top3 0.63533
wandb:                                 mol1_top5 0.76269
wandb:                                  mol2_acc 0.56441
wandb:                                 mol2_loss 1.69135
wandb:                                 mol2_top3 0.77284
wandb:                                 mol2_top5 0.8599
wandb:                                  mol3_acc 0.36212
wandb:                                 mol3_loss 2.39015
wandb:                                 mol3_top3 0.6326
wandb:                                 mol3_top5 0.74832
wandb:                                  mol4_acc 0.42445
wandb:                                 mol4_loss 2.08651
wandb:                                 mol4_top3 0.69539
wandb:                                 mol4_top5 0.80757
wandb:                                  mol5_acc 0.78742
wandb:                                 mol5_loss 1.02609
wandb:                                 mol5_top3 0.8754
wandb:                                 mol5_top5 0.9289
wandb:                           trust_labelling False

In [ ]:
# super class fgi

In [ ]:
# super class reductions

In [ ]:
# mid split, gao model


In [ ]:
# Kobi code
DATASETS = ["with_trust_with_map","with_trust_no_map","no_trust_with_map", "no_trust_no_map"]
lines = ["Solvents", "Agents", "Solvents \& Agents"]
top_n = 3
for dataset in DATASETS:
    filters = {
        "state": "finished",
        "config.output_folder_path": f"models/{dataset}",
        "config.random_seed": 54321,
        "config.train_fraction": 1.0,
        "config.train_mode": 0, # Teacher forcing
        "config.dataset_version": {"$in": ["v4", "v5"]}
    }
    runs = api.runs(
        f"{wandb_entity}/{wandb_project}",
        filters=filters
    )
    run = runs[0]
    if len(runs)>0:
        for r in runs:
            if run.config["dataset_version"] == "v5":
                run = r
                break

    # Get model solvent, agent and overall accuracy
    test_best = run.summary["test_best"]
    solvent_accuracy = test_best[f"solvent_accuracy_top{top_n}"]
    agent_accuracy = test_best[f"three_agents_accuracy_top{top_n}"]
    overall_accuracy = test_best[f"overall_accuracy_top{top_n}"]

    # Get frequency informed solvent, agent and overall accuracy
    fi_solvent_accuracy = run.summary[f"frequency_informed_solvent_accuracy_top_{top_n}"]
    fi_agent_accuracy = run.summary[f"frequency_informed_agent_accuracy_top_{top_n}"]
    fi_overall_accuracy = run.summary[f"frequency_informed_overall_accuracy_top_{top_n}"]

    # Improvement
    solvent_improvement = (solvent_accuracy-fi_solvent_accuracy)/(1-fi_solvent_accuracy)
    solvent_improvement_color = "lessgreen" if solvent_improvement>0 else "red"
    agent_improvement = (agent_accuracy-fi_agent_accuracy)/(1-fi_agent_accuracy)
    agent_improvement_color = "lessgreen" if agent_improvement>0 else "red"
    overall_improvement = (overall_accuracy-fi_overall_accuracy)/(1-fi_overall_accuracy)
    overall_improvement_color = "lessgreen" if overall_improvement>0 else "red"

    # Create table lines
    lines[0] += f" & {fi_solvent_accuracy*100:.0f} // {solvent_accuracy*100:.0f} // \\textcolor{{{solvent_improvement_color}}}{{{solvent_improvement*100:.0f}\%}} "
    lines[1] += f" & {fi_agent_accuracy*100:.0f} // {agent_accuracy*100:.0f} // \\textcolor{{{agent_improvement_color}}}{{{agent_improvement*100:.0f}\%}} "
    lines[2] += f" & {fi_overall_accuracy*100:.0f} // {overall_accuracy*100:.0f} // \\textcolor{{{overall_improvement_color}}}{{{overall_improvement*100:.0f}\%}} "
print("\\\\ \n".join(lines) + "\\\\")